In [81]:
import pandas as pd
import numpy as np

from time import sleep
from random import uniform

import concurrent.futures
from functools import partial, reduce    

from yahoofinancials import YahooFinancials
from warnings import warn
from datetime import date, timedelta

import scipy.optimize as sco

from prophet import Prophet

# Descarga de datos

In [2]:
def extraction(ticker: object, 
               sdate: object,
               edate: object,
               frequency: object,
               volatility: bool = False) -> pd.DataFrame:
    '''
    Esta función extrae el ticker con las fechas y la frecuencia especificadas. Devuelve un dataframe con
    tres columnas: la fecha, los precios de cierre (precio ajustado), y una medida de volatilidad del periodo.
    '''
    #Consulta el ticker
    stock = YahooFinancials(ticker)
    #Se trae las fechas históricas con la frecuencia requerida
    price = stock.get_historical_price_data(start_date = sdate, 
                                end_date = edate, 
                                time_interval = frequency)

    #Toma los precios
    df_price = pd.DataFrame(price[ticker]['prices'])
    #Intenta calcular precio y volatilidad
    try:
        #Toma el precio de cierre ajustado
        df_price['{}_price'.format(ticker)] = df_price['adjclose']
        if volatility is True:
            #Con precios máximos, mínimos y cierre calcula volatilidad
            df_price['{}_volatility'.format(ticker)] = (df_price['high'] - df_price['low'])/df_price['close']
    #Si no lo logra devuelve Null
    except:
        warn('Error en la moneda {}'.format(ticker))
        df_price['{}_price'.format(ticker)] = None
        if volatility is True:
            #Con precios máximos, mínimos y cierre calcula volatilidad
            df_price['{}_volatility'.format(ticker)] = None
        
        df_price['formatted_date'.format(ticker)] = None
    #Al final selecciona las columnas finales
    finally:
        #Formatea fecha
        df_price.drop(columns = df_price.columns.difference(['formatted_date',
                                               '{}_price'.format(ticker),
                                               '{}_volatility'.format(ticker)]), inplace=True)
    
    return df_price

In [58]:
fecha_inicial = '2015-01-01'
fecha_final = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d")
freq = 'daily' 

#Extrae los tickers
tickers = ['BTC-USD', 'ETH-USD', 'DOT-USD', 'BNB-USD', 'SOL-USD', 'ADA-USD', 'LUNA-USD', 
           'XRP-USD', 'AVAX-USD', 'MATIC-USD', 'LINK-USD', 'ALGO-USD']
#Ponderadores
w = np.array([0.3, 0.2, 0.08, 0.07, 0.06, 0.06, 0.05, 0.05, 0.05, 0.03, 0.02, 0.03]).reshape(1, -1)

#Fija los argumentos extra de la función
extract_ticker = partial(extraction,
                         sdate = fecha_inicial,
                         edate = fecha_final,
                         frequency = freq,
                         volatility = False)

with concurrent.futures.ThreadPoolExecutor() as executor:
    all_cryptos = executor.map(extract_ticker, tickers)

#Reduciremos el mapa de data frames en un data 
cryptos_reduce = partial(pd.merge, on='formatted_date', how='outer')                                                              
#Reduce el mapa de dataframes en un solo dataframe
df = reduce(cryptos_reduce, all_cryptos) 

#Se trae la tasa "libre de riesgo"
df_rf = extraction('^TNX', fecha_inicial, fecha_final, freq).dropna()

# Preprocesamiento

In [59]:
thr_nulls = 0.9

#Depuramos los datos
df_filter = df.loc[:, df.isnull().sum(axis = 0)/df.shape[0] < thr_nulls].copy()
df_filter.dropna(axis = 0, how = 'any', inplace = True)

#Fijamos índices
df_filter = df_filter.set_index(pd.to_datetime(df_filter['formatted_date']))
df_filter.drop(columns = ['formatted_date'], inplace = True)

df_rf = df_rf.set_index(pd.to_datetime(df_rf['formatted_date']))
df_rf.drop(columns = ['formatted_date'], inplace = True)

In [82]:
def prophet_trend(str_crypto: object,
                 future_days: int = 30) -> pd.Series:
    df_prophet = df_filter[str_crypto].reset_index().copy()
    df_prophet.columns = ['ds', 'y']

    prophet = Prophet()
    prophet.fit(df_prophet)

    future = prophet.make_future_dataframe(periods=future_days)
    forecast = prophet.predict(future)

    return forecast.iloc[-future_days:, 1]

In [79]:
predicted_cryptos = df_filter.columns
with concurrent.futures.ThreadPoolExecutor() as executor:
    all_trends = executor.map(prophet_trend, predicted_cryptos)
    
df_trends = pd.DataFrame(all_trends).T
df_trends.columns = predicted_cryptos

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to o

C:\Users\david\anaconda3\envs\time_series\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\david\anaconda3\envs\time_series\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\david\anaconda3\envs\time_series\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\david\anaconda3\envs\time_series\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [87]:
#Se calculan cambios porcentuales
df_filter = df_trends.copy()

df_filter = df_filter.apply(lambda x: x.pct_change(), axis = 0)
df_filter.dropna(inplace = True)

df_filter.mean(axis = 0).sort_values()

ADA-USD_price     -0.012792
SOL-USD_price     -0.001369
LINK-USD_price    -0.001310
BTC-USD_price     -0.001151
ALGO-USD_price    -0.000603
XRP-USD_price     -0.000476
LUNA-USD_price    -0.000136
DOT-USD_price      0.000391
ETH-USD_price      0.000481
BNB-USD_price      0.001025
MATIC-USD_price    0.001694
AVAX-USD_price     0.002992
dtype: float64